# Creating the Distance Matrix

In order to calculate the access metrics, we need a distance matrix of each supply point to each demand point. In our case, from the census tract centroid to each school. 

Calculating the distance between every single pair of supply and demand points is computationally inviable, so we define a threshold for a maximum distance and we split the calculation into chunks. In practice, we divide our dataframe based on the microregions of Brazil and calculate the distance matrix of each one separately. The benefit is that we don't have to check the distance for two points that are hundreds of miles away. The downside is that with this method we ignore that people can cross microregions to go to school. However, I consider that this scenario is extremely uncommon. 

We will use the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula) to calculate the distance between two points on a sphere. The Haversine formula is a special case of the [Vincenty formula](https://en.wikipedia.org/wiki/Vincenty%27s_formulae), which is more accurate but also more computationally expensive. The Haversine formula is a good approximation for short distances, which is the case for our analysis.

W=

In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [4]:

# importing basic dataset of the census tracts
geo_school_census_df = pd.read_csv('/Users/feliphlvo/Documents/Minerva/Capstone/data/Local/school_census.csv', index_col=0)
geo_dem_census_df = pd.read_csv('/Users/feliphlvo/Documents/Minerva/Capstone/data/Local/dem_census.csv', index_col=0)

In [5]:
# Only public schools with at least one high school class and regions with at least one high-school aged person
geo_dem_census_df = geo_dem_census_df[geo_dem_census_df["n_people_15to17"] > 0]
geo_school_census_df = geo_school_census_df[(geo_school_census_df["n_classes"] > 0) & (geo_school_census_df["admin_type"] != 4.0)]

In [141]:
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')
def distance_matrix(demand_df, supply_df, demand_index, supply_index, name = "euclidean", threshold = 10000):   
    """""
    This function calculates the distance matrix between every pair of points in two geodataframes up to a threshold.

    Inspired by create_euclidean_distances in the Access package
    """

    # Reset the index so that the geoids are accessible
    df1 = demand_df.rename_axis("origin").reset_index()
    df2 = supply_df.rename_axis("dest").reset_index()

    # Convert to centroids if so-specified
    df1.set_geometry(df1.centroid, inplace=True)
    df2.set_geometry(df2.centroid, inplace=True)
    # Calculate the distances.
    if (df1.geom_type == "Point").all() & (df2.geom_type == "Point").all():
        # If both geometries are point types, merge on a temporary dummy column
        df1["temp"] = 1
        df2["temp"] = 1
        df1and2 = df1[["temp", "geometry", "origin"]].merge(
            df2[["temp", "geometry", "dest"]].rename(columns={"geometry": "geomb"})
        )
        df1and2.drop("temp", inplace=True, axis=1)
        df1and2[name] = df1and2.distance(df1and2.set_geometry("geomb"))
    else:
        # Execute an sjoin for non-point geometries, based upon a buffer zone
        df1and2 = gpd.sjoin(
            df1,
            df2.rename(columns={"geometry": "geomb"}).set_geometry(
                df2.buffer(threshold)
            ),
        )
        df1and2[name] = df1and2.distance(df1and2.set_geometry("geomb"))

    # Add it to the cost df.
    df1and2 = df1and2[df1and2[name] < threshold]
    df1and2 = df1and2.reset_index()[["origin", "dest", "euclidean"]]

    return df1and2


def chunked_distance_matrix(demand_df, supply_df, demand_index, supply_index, subset_column, name = "euclidean", threshold = 10000):
    '''''
    This function calculates the distance matrix between every pair of points in two geodataframes up to a threshold.
    It calculates distances separately for each subset to avoid memory issues.
    '''
    demand_df = demand_df.copy()
    supply_df = supply_df.copy()
    
    demand_df.set_index(demand_index, inplace=True)
    supply_df.set_index(supply_index, inplace=True)

    subset_values = demand_df[subset_column].unique()
    # Initialize distance matrix df
    distance_matrix_df = pd.DataFrame()
    for value in tqdm(subset_values):
        #print("Processing " + str(value))
        # Subset the dataframes
        demand_subset = demand_df[demand_df[subset_column] == value]
        supply_subset = supply_df[supply_df[subset_column] == value]
        # Calculate the distance matrix
        distance_matrix_df = distance_matrix_df.append(
            distance_matrix(demand_subset, supply_subset,  demand_index, supply_index, name, threshold)
        )
    return distance_matrix_df

In [143]:
dist_matrix = chunked_distance_matrix(geo_dem_census_df, geo_school_census_df, demand_index="sector_id",
                                      supply_index="school_id", subset_column="microregion_id", name="euclidean", threshold=20000)

100%|██████████| 559/559 [07:06<00:00,  1.31it/s]


In [147]:
dist_matrix.head()

,origin,dest,euclidean
0,170025105000002,17010535,13894.563390
1,170320605000005,17014298,16119.745299
2,170390905000005,17011736,12566.395935
3,170600105000005,17011787,17742.292778
4,170600105000005,17014298,18544.963646


In [146]:
# Write distance matrix to csv
dist_matrix.to_csv('/Users/feliphlvo/Documents/Minerva/Capstone/data/Local/dist_matrix.csv')

In [45]:
# Unique values of city_id
geo_dem_census_df["city_id"].unique()

array([1700251., 1700301., 1700400., ..., 4207858., 2413805., 2700805.])

In [40]:
# Subset DFs
subset_city = 1100015.0
subset_census = geo_dem_census_df[geo_dem_census_df["city_id"] == subset_city]
subset_schools = geo_school_census_df[geo_school_census_df["city_id"] == subset_city]

In [44]:
distance_matrix(subset_census, subset_schools, name = "euclidean", threshold = 10000)

/Users/feliphlvo/opt/anaconda3/envs/cap_env/lib/python3.10/site-packages/geopandas/array.py:546: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return _unary_geo("centroid", self)
/Users/feliphlvo/opt/anaconda3/envs/cap_env/lib/python3.10/site-packages/geopandas/geoseries.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = from_shapely(s.values)


,geometry,origin,geomb,dest,euclidean
0,POINT (2886898.637 8672132.565),10650,POINT (2886759.309 8672190.023),8,150.710340
1,POINT (2886898.637 8672132.565),10650,POINT (2887396.839 8672498.304),12,618.038247
2,POINT (2886898.637 8672132.565),10650,POINT (2885894.346 8671383.156),27,1253.081621
15,POINT (2886629.031 8671143.039),108481,POINT (2886759.309 8672190.023),8,1055.057393
16,POINT (2886629.031 8671143.039),108481,POINT (2887396.839 8672498.304),12,1557.649721
...,...,...,...,...,...
148,POINT (2886438.056 8672714.282),284272,POINT (2887396.839 8672498.304),12,982.807917
149,POINT (2886438.056 8672714.282),284272,POINT (2885894.346 8671383.156),27,1437.886132
156,POINT (2886044.618 8672622.476),306095,POINT (2886759.309 8672190.023),8,835.343847
157,POINT (2886044.618 8672622.476),306095,POINT (2887396.839 8672498.304),12,1357.910520
